In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
dir_ = '../../../../data/'
group_dir_ = os.path.join(dir_, 'groups/low')

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
all_user = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

file_name = 'normalized_popularity_filter_track_5_user_100.pkl'
pop_count = pd.read_pickle(os.path.join(dir_, file_name))

num_user = len(svd['uid'].unique())
num_user

953

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [5]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,9,2.243909,1
1,0,15,2.160610,1
2,0,22,2.145235,1
3,0,28,2.204445,1
4,0,33,2.166677,1


In [6]:
pop_count = pop_count.sort_values(by=['count'],  ascending=False)
pop_count[:10]

,tid,count,rating
70,70,17557,2980.75
1102,1102,6926,1484.25
83,83,6571,1796.75
13496,13496,6299,1669.75
952,952,6234,1527.50
2519,2519,6001,1772.25
4008,4008,5676,1008.25
5521,5521,5658,1727.25
3900,3900,5627,1475.50
2241,2241,5571,1183.75


In [7]:
# min_max
max_rating = pop_count.iloc[0]['count']
pop_normalized = pop_count.copy()
pop_normalized = pop_normalized[['tid', 'count']]
pop_normalized['rating'] = pop_normalized['count']
pop_normalized['rating'] /= max_rating
pop_normalized['rating'] *= 5
pop_normalized[:10]

,tid,count,rating
70,70,17557,5.000000
1102,1102,6926,1.972433
83,83,6571,1.871333
13496,13496,6299,1.793871
952,952,6234,1.775360
2519,2519,6001,1.709005
4008,4008,5676,1.616449
5521,5521,5658,1.611323
3900,3900,5627,1.602495
2241,2241,5571,1.586547


In [8]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [9]:
NOVs = []
COVs = []
HitNOVs = []
HitCOVs = []
for lambda_ in tqdm(lambdas):
    top_n_size = 50
    top_n_lists_g = []
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []
        top_n_items = pd.DataFrame()

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = pop_normalized.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
            
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
                
        top_n_lists_g.append(top_n_lists)   
        top_n_items_g.append(top_n_items)
        
    COV = []    
    NOV = []
    HitNOV = []
    HitCOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        top_n_items = top_n_items_g[i]

        nov = 0
        hitNov = 0
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):
            group = groups[j]
            top_n_list = top_n_lists[j]
            
            test_g = pd.DataFrame()
            for uid in group:
                t = test[test['uid'] == uid]
                if len(test_g) == 0:
                    test_g = t
                else:
                    test_g = test_g.set_index('tid').add(t.set_index('tid'), fill_value=0).reset_index()
            test_g['rating'] /= len(group)
            
            for k in range(top_n_size):
                tid = top_n_list[k]
                t = test_g[test_g['tid'] == tid]
                nov += (1-pop_dict[tid])
                if len(t) > 0:
                    hits[int(tid)] = True
                    hitNov += (1-pop_dict[tid])
        NOV.append(nov/top_n_size/len(groups))
        COV.append(len(top_n_items)/len(df['tid'].unique()))
        HitNOV.append(hitNov/top_n_size/len(groups))
        HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
    NOVs.append(NOV)
    COVs.append(COV)
    HitNOVs.append(HitNOV)
    HitCOVs.append(HitCOV)
    
for NOV in NOVs:
    print(NOV)
print()
for HitNOV in HitNOVs:
    print(HitNOV)
print()  
for COV in COVs:
    print(COV)
print()    
for HitCOV in HitCOVs:
    print(HitCOV)


[0.549066107030465, 0.5490661070304361, 0.5490661070304306, 0.5490661070304298, 0.5490661070304279, 0.5490661070304262, 0.549066107030427]
[0.5576191818808309, 0.5545695486612999, 0.5526852803409752, 0.5517962415339128, 0.551472542847148, 0.5508580192105865, 0.5510340859086725]
[0.5714065801827325, 0.5690604314742637, 0.5649750651767071, 0.5609068650831516, 0.5588182056663163, 0.556657249710762, 0.5572419862573172]
[0.5889132147207058, 0.5795847604717363, 0.5774766607890436, 0.5744510159305564, 0.5700590241343123, 0.5678381359808414, 0.5669112818603368]
[0.5974721029668945, 0.5968827997650649, 0.5881277787994499, 0.5834589335114018, 0.581554739419377, 0.5791589313100313, 0.5760572724503243]
[0.6143383338226462, 0.6057387792274711, 0.6081331876548276, 0.6004651976215464, 0.5940709164043366, 0.5930454435386208, 0.5896327387198298]
[0.6192038554441162, 0.6234615613059129, 0.6190936921928574, 0.6180132913606159, 0.6173469744665953, 0.6150961874781369, 0.6070317841789916]
[0.62112024137220